# Binary Classification

In [1]:
from bnlp import BengaliWord2Vec
import numpy as np

bwv = BengaliWord2Vec()
model_path = "bangla_word2vec/bnwiki_word2vec.model"


def vectorize_word(word):
    return bwv.generate_word_vector(model_path, word)
vectorize_word("নৃগোষ্ঠী")

/opt/conda/lib/python3.7/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


array([ 2.3823591e-01,  6.1981279e-01, -1.9406676e-01,  4.1382647e-01,
        6.7421144e-01,  1.4040026e-01,  1.6097938e+00, -6.3083977e-01,
       -1.0239607e+00, -5.4499447e-02, -9.9497877e-02, -3.1477413e-01,
        1.7324188e-01,  1.8793076e-01,  7.9062946e-02,  5.0363833e-01,
        1.4413389e+00,  1.2055126e-01,  8.0126077e-02,  2.2117047e-01,
       -3.6095098e-01, -5.7067740e-01,  2.4038902e-01, -9.4271898e-01,
        5.5764532e-01, -2.8166807e-01,  4.3514800e-01,  3.5518160e-01,
       -2.5896186e-01,  1.4379877e-01, -5.8390427e-01,  8.1498146e-01,
        2.8396347e-01,  2.5972643e-01,  3.5667416e-01, -2.3208426e-01,
       -1.1156581e-02,  2.1004668e-01, -3.4496439e-01,  7.1494210e-01,
        6.2013745e-01, -1.5306453e+00,  1.2590113e-01,  4.3290872e-02,
       -3.9522484e-01,  1.8227801e+00,  8.5033786e-01, -1.2005626e+00,
       -7.1115762e-01,  1.8970840e-01,  5.3540075e-01,  1.1116323e+00,
       -5.1466793e-01,  5.6390721e-01, -1.5165229e+00, -9.0429567e-02,
      

In [37]:
import string
def preprocess_word(word): ## hANDLE DARI ।
  translate_table = dict((ord(char), None) for char in string.punctuation)   
  word =  word.translate(translate_table)
  return word.replace("।", "")

print(preprocess_word("পর,"))

পর


In [4]:
import string
def preprocess_word(word): ## hANDLE DARI ।
  translate_table = dict((ord(char), None) for char in string.punctuation)   
  word =  word.translate(translate_table)
  return word.replace("।", "")

print(preprocess_word("পর,"))

পর


In [46]:
def getDatasetFromIBOFile(filename): #nlp_hackathon_bd_2023/data/dev.txt"
    f = open(filename, "r")
    content = f.read()
    
    lines = content.split("\n")
    
    dataset = []
    for line in lines:
        parts = line.split(" _ _ ")
        word = parts[0]
        word = preprocess_word(word)
        label = parts[len(parts)-1]

        dataset.append([word,label])
    return dataset


In [47]:
def encodeClass(Y):
    classes = [0,0] # Yes, No(Not NER)
    
    new_y = []
    for cls in Y:
        if cls=='O': new_y.append(0)
        else: new_y.append(1)
    return new_y
                
# encodeClass(Y)

In [48]:
def preprocess_dataset(dataset):
    X = []
    Y = []
    for row in dataset:
      word = row[0]
      label = row[1]

      print(word)

      X.append(vectorize_word(word))
      Y.append(label)
    
    Y = encodeClass(Y)
    
    Y = np.array(Y)
    X = np.array(X)
    return X, Y
    

In [50]:
print(dataset)
print(Y)

[['তিনি', 'O'], ['যুবক', 'O'], ['হিসেবে', 'O'], ['শেফিল্ড', 'B-GRP'], ['বুধবার', 'I-GRP'], ['এফসি', 'I-GRP'], ['যোগদান', 'O'], ['করেন', 'O'], ['এবং', 'O'], ['১৯৫১', 'O'], ['সালে', 'O']]
[0 0 0 1 1 1 0 0 0 0 0]


In [49]:
DATA_SIZE = 11
dataset = getDatasetFromIBOFile("nlp_hackathon_bd_2023/data/dev.txt")[:DATA_SIZE]
X, Y = preprocess_dataset(dataset)

তিনি
যুবক
হিসেবে
শেফিল্ড
বুধবার
এফসি
যোগদান
করেন
এবং
১৯৫১
সালে


### Model!

In [23]:
import numpy as np
import random

from random import randint
from sklearn.utils import shuffle
from sklearn.preprocessing import MinMaxScaler

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy

from tensorflow.keras import layers

In [79]:
binary_model = Sequential([
    layers.Dense(256, input_dim=100, activation="relu", name="input"),
    layers.Dense(128, activation="relu", name="layer1"),
    #layers.Dense(64, activation="relu", name="layer2"),
    layers.Dense(2, activation="softmax", name="output")
])
binary_model.compile(optimizer=Adam(learning_rate=0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
binary_model.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (Dense)                (None, 256)               25856     
_________________________________________________________________
layer1 (Dense)               (None, 128)               32896     
_________________________________________________________________
output (Dense)               (None, 2)                 258       
Total params: 59,010
Trainable params: 59,010
Non-trainable params: 0
_________________________________________________________________


In [80]:
binary_model.fit(x=X, y=Y, batch_size=10, epochs=30, verbose=2, validation_split=0.1)

Train on 9 samples, validate on 2 samples
Epoch 1/30


InvalidArgumentError: Received a label value of 5 which is outside the valid range of [0, 2).  Label values: 3 4 0 1 5 1 4 2 2
	 [[{{node loss_10/output_loss/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits}}]]

In [53]:
model.predict(X)

array([[0.8984386 , 0.10156144],
       [0.90100056, 0.09899945],
       [0.9836235 , 0.01637647],
       [0.10870043, 0.8912995 ],
       [0.21083497, 0.789165  ],
       [0.15912646, 0.84087354],
       [0.9734622 , 0.02653786],
       [0.9865841 , 0.01341592],
       [0.9514842 , 0.04851579],
       [0.7769665 , 0.2230335 ],
       [0.62828356, 0.3717165 ]], dtype=float32)

# Multi-class Classification

In [65]:
def getNERDatasetFromIBOFile(filename): #nlp_hackathon_bd_2023/data/dev.txt"
    f = open(filename, "r")
    content = f.read()
    
    lines = content.split("\n")
    
    dataset = []
    for line in lines:
        parts = line.split(" _ _ ")
        word = parts[0]
        word = preprocess_word(word)
        label = parts[len(parts)-1]

        if label!='O' and word!='': dataset.append([word,label])
    return dataset


In [120]:
global_classes = [] # Global
def encodeMultiClass(Y):
    for val in Y:
        if val not in global_classes:
            global_classes.append(val)
            
    new_y = []
    for val in Y:
        for idx, cls in enumerate(global_classes):
            if cls==val: 
                new_y.append(idx)
                break
    return new_y

In [69]:
def preprocess_dataset(dataset):
    X = []
    Y = []
    for row in dataset:
      word = row[0]
      label = row[1]

      print(word)

      X.append(vectorize_word(word))
      Y.append(label)
    
    Y = encodeMultiClass(Y)
    
    Y = np.array(Y)
    X = np.array(X)
    return X, Y
    

In [144]:
DATA_SIZE = 11
dataset = getNERDatasetFromIBOFile("nlp_hackathon_bd_2023/data/dev.txt")[:DATA_SIZE]
X, Y = preprocess_dataset(dataset)

শেফিল্ড
বুধবার
এফসি
চাটনি
পোলিশ
লিথুয়ানিয়ান
কমনওয়েলথ
আলোকচিত্রগ্রাহী
ফিল্ম
ভ্যারাইটি
জাপানের


In [71]:
print(Y)

[0 1 1 2 3 4 4 2 5 6 0]


In [127]:
NUM_OF_CLASSES = 7
multiclass_model = Sequential([
    layers.Dense(256, input_dim=100, activation="relu", name="input"),
    layers.Dense(128, activation="relu", name="layer1"),
    #layers.Dense(64, activation="relu", name="layer2"),
    layers.Dense(NUM_OF_CLASSES, activation="softmax", name="output")
])
multiclass_model.compile(optimizer=Adam(learning_rate=0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
multiclass_model.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (Dense)                (None, 256)               25856     
_________________________________________________________________
layer1 (Dense)               (None, 128)               32896     
_________________________________________________________________
output (Dense)               (None, 7)                 903       
Total params: 59,655
Trainable params: 59,655
Non-trainable params: 0
_________________________________________________________________


In [78]:
multiclass_model.fit(x=X, y=Y, batch_size=10, epochs=30, verbose=2, validation_split=0.1)

Train on 9 samples, validate on 2 samples
Epoch 1/30
9/9 - 0s - loss: 2.1973 - acc: 0.1111 - val_loss: 1.7508 - val_acc: 0.0000e+00
Epoch 2/30
9/9 - 0s - loss: 2.1352 - acc: 0.2222 - val_loss: 1.7692 - val_acc: 0.0000e+00
Epoch 3/30
9/9 - 0s - loss: 2.0745 - acc: 0.2222 - val_loss: 1.7876 - val_acc: 0.0000e+00
Epoch 4/30
9/9 - 0s - loss: 2.0147 - acc: 0.2222 - val_loss: 1.8065 - val_acc: 0.0000e+00
Epoch 5/30
9/9 - 0s - loss: 1.9556 - acc: 0.2222 - val_loss: 1.8249 - val_acc: 0.0000e+00
Epoch 6/30
9/9 - 0s - loss: 1.8979 - acc: 0.2222 - val_loss: 1.8434 - val_acc: 0.0000e+00
Epoch 7/30
9/9 - 0s - loss: 1.8423 - acc: 0.2222 - val_loss: 1.8618 - val_acc: 0.0000e+00
Epoch 8/30
9/9 - 0s - loss: 1.7888 - acc: 0.2222 - val_loss: 1.8801 - val_acc: 0.0000e+00
Epoch 9/30
9/9 - 0s - loss: 1.7371 - acc: 0.3333 - val_loss: 1.8984 - val_acc: 0.0000e+00
Epoch 10/30
9/9 - 0s - loss: 1.6864 - acc: 0.3333 - val_loss: 1.9167 - val_acc: 0.0000e+00
Epoch 11/30
9/9 - 0s - loss: 1.6367 - acc: 0.3333 - val_l

# Make prediction using these two layers

In [145]:
def predictClass(word, binary_classifier, multiclass_classifier):
    x = vectorize_word(word).reshape(1,100)
    predictions = binary_classifier.predict(x)
    print(predictions)
    if predictions[0][0] > predictions[0][1]: return "O"

    cls = list(multiclass_classifier.predict(x))

    print(cls)
    max_value = max(cls[0])
    max_index = list(cls[0]).index(max_value)
    print(max_value, max_index)

    
    return global_classes[max_index]
 

In [146]:
predictClass("কমনওয়েলথ", binary_model, multiclass_model)

[[0.13426256 0.86573744]]
[array([0.22747843, 0.04951087, 0.10875148, 0.0873021 , 0.15978889,
       0.02095048, 0.34621775], dtype=float32)]
0.34621775 6


'B-CORP'